In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import math
from UnarySim.sw.kernel.tanh import tanhP1
from UnarySim.sw.kernel.mul import UnaryMul_int
from UnarySim.sw.stream.gen import RNG, SourceGen, BSGen
from UnarySim.sw.metric.metric import ProgressiveError
import matplotlib.pyplot as plt
import time
import math
import numpy as np
global perfect_result_RMSE
global perfect_result_MAE

In [3]:
def tanh_comb_test(bw=8, mode="unipolar", rng="Sobol"):
    
    global perfect_result_RMSE
    global perfect_result_MAE
    
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    total_cnt = 100
    bitwidth = bw
    btype = torch.float
    rtype=torch.float
    stype=torch.float

    print("========================================================")
    print(mode)
    print("========================================================")
    # all input values are non-negative
    low_bound = 0
    if mode is "unipolar":
        up_bound = 2**bitwidth
    elif mode is "bipolar":
        low_bound = 0
        up_bound = 2**(bitwidth-1)

    input_list = []
    for input_val in range(low_bound, up_bound+1, 1):
        input_list.append(input_val)

    input = torch.tensor(input_list).type(torch.float).div(up_bound).to(device)
    output = torch.tanh(input)
    
    result_pe_total_normal = []
    for rand_idx in range(1, total_cnt+1):
        
        outputPE = ProgressiveError(output, mode=mode).to(device)
        inputPE  = ProgressiveError(input,  mode=mode).to(device)
        
        dut_tanh_comb = tanhP1(mode=mode, 
                               rng=rng, 
                               rng_dim=rand_idx, 
                               rng_width=bitwidth).to(device)
        inputSRC = SourceGen(input, bitwidth, mode=mode, rtype=rtype)().to(device)
        inputRNG = RNG(bitwidth, rand_idx, rng, rtype)().to(device)
        inputBS = BSGen(inputSRC, inputRNG, stype).to(device)
        with torch.no_grad():
            start_time = time.time()
            for i in range(2**bitwidth):
                input_bs = inputBS(torch.tensor([i]))
                inputPE.Monitor(input_bs)

                output_bs = dut_tanh_comb(input_bs)
                outputPE.Monitor(output_bs)
        
        # get the result for different rng
        result_pe = outputPE()[1].cpu().numpy()
        result_pe_total_normal.append(result_pe)        
    
    # get the result for different rng
    result_pe_total_normal = np.array(result_pe_total_normal)
    #######################################################################
    # check the error of all simulation
    #######################################################################
    print("RMSE:{:1.4}".format(math.sqrt(np.mean(result_pe_total_normal**2))))
    perfect_result_RMSE = math.sqrt(np.mean(result_pe_total_normal**2))
    print("MAE: {:1.4}".format(np.mean(np.abs(result_pe_total_normal))))
    perfect_result_MAE = np.mean(np.abs(result_pe_total_normal))
    print("bias:{:1.4}".format(np.mean(result_pe_total_normal)))
    print("max: {:1.4}".format(np.max(result_pe_total_normal)))
    print("min: {:1.4}".format(np.min(result_pe_total_normal)))

In [4]:
tanh_comb_test(8, "unipolar", "Sobol")

unipolar
RMSE:0.01742
MAE: 0.01282
bias:-0.008678
max: 0.0799
min: -0.0672


In [5]:
def tanh_comb_test_intermittent_perfect(bw=8, mode="unipolar", rng="Sobol", power_off=128):
    
    global perfect_result_RMSE
    global perfect_result_MAE
    
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    total_cnt = 100
    bitwidth = bw
    btype = torch.float
    rtype=torch.float
    stype=torch.float

    print("========================================================")
    print(mode)
    print("========================================================")
    # all input values are non-negative
    low_bound = 0
    if mode is "unipolar":
        up_bound = 2**bitwidth
    elif mode is "bipolar":
        low_bound = 0
        up_bound = 2**(bitwidth-1)

    input_list = []
    for input_val in range(low_bound, up_bound+1, 1):
        input_list.append(input_val)

    input = torch.tensor(input_list).type(torch.float).div(up_bound).to(device)
    output = torch.tanh(input)
    
    result_pe_total_normal = []
    for rand_idx in range(1, total_cnt+1):
        
        outputPE = ProgressiveError(output, mode=mode).to(device)
        inputPE  = ProgressiveError(input,  mode=mode).to(device)
        
        dut_tanh_comb = tanhP1(mode=mode, 
                               rng=rng, 
                               rng_dim=rand_idx, 
                               rng_width=bitwidth).to(device)
        inputSRC = SourceGen(input, bitwidth, mode=mode, rtype=rtype)().to(device)
        inputRNG = RNG(bitwidth, rand_idx, rng, rtype)().to(device)
        inputBS = BSGen(inputSRC, inputRNG, stype).to(device)
        with torch.no_grad():
            start_time = time.time()
            for i in range(0,power_off):
                input_bs = inputBS(torch.tensor([i]))
                inputPE.Monitor(input_bs)

                output_bs = dut_tanh_comb(input_bs)
                outputPE.Monitor(output_bs)
            for i in range(power_off,2**bitwidth):
                input_bs = inputBS(torch.tensor([i]))
                inputPE.Monitor(input_bs)

                output_bs = dut_tanh_comb(input_bs)
                outputPE.Monitor(output_bs)
        
        # get the result for different rng
        result_pe = outputPE()[1].cpu().numpy()
        result_pe_total_normal.append(result_pe)        
    
    # get the result for different rng
    result_pe_total_normal = np.array(result_pe_total_normal)
    #######################################################################
    # check the error of all simulation
    #######################################################################
    print("RMSE:{:1.4}".format(math.sqrt(np.mean(result_pe_total_normal**2))))
    print("MAE: {:1.4}".format(np.mean(np.abs(result_pe_total_normal))))
    print("bias:{:1.4}".format(np.mean(result_pe_total_normal)))
    print("max: {:1.4}".format(np.max(result_pe_total_normal)))
    print("min: {:1.4}".format(np.min(result_pe_total_normal)))
    print("RMSE_error from perfect:",(math.sqrt(np.mean(result_pe_total_normal**2))-perfect_result_RMSE)/perfect_result_RMSE)
    print("MAE_error from perfect:",(np.mean(np.abs(result_pe_total_normal))-perfect_result_MAE)/perfect_result_MAE)

In [6]:
tanh_comb_test_intermittent_perfect(8, "unipolar", "Sobol")

unipolar
RMSE:0.01742
MAE: 0.01282
bias:-0.008678
max: 0.0799
min: -0.0672
RMSE_error from perfect: 0.0
MAE_error from perfect: 0.0


In [10]:
def tanh_comb_test_intermittent_indx(bw=8, mode="unipolar", rng="Sobol", power_off=128):
    
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    total_cnt = 100
    bitwidth = bw
    btype = torch.float
    rtype=torch.float
    stype=torch.float

    print("========================================================")
    print(mode)
    print("========================================================")
    # all input values are non-negative
    low_bound = 0
    if mode is "unipolar":
        up_bound = 2**bitwidth
    elif mode is "bipolar":
        low_bound = 0
        up_bound = 2**(bitwidth-1)

    input_list = []
    for input_val in range(low_bound, up_bound+1, 1):
        input_list.append(input_val)

    input = torch.tensor(input_list).type(torch.float).div(up_bound).to(device)
    output = torch.tanh(input)
    
    result_pe_total_normal = []
    for rand_idx in range(1, total_cnt+1):
        
        outputPE = ProgressiveError(output, mode=mode).to(device)
        inputPE  = ProgressiveError(input,  mode=mode).to(device)
        
        dut_tanh_comb = tanhP1(mode=mode, 
                               rng=rng, 
                               rng_dim=rand_idx, 
                               rng_width=bitwidth).to(device)
        inputSRC = SourceGen(input, bitwidth, mode=mode, rtype=rtype)().to(device)
        inputRNG = RNG(bitwidth, rand_idx, rng, rtype)().to(device)
        inputBS = BSGen(inputSRC, inputRNG, stype).to(device)
        with torch.no_grad():
            start_time = time.time()
            for i in range(0,power_off):
                input_bs = inputBS(torch.tensor([i]))
                inputPE.Monitor(input_bs)

                output_bs = dut_tanh_comb(input_bs)
                outputPE.Monitor(output_bs)
                
            dut_tanh_comb = tanhP1(mode=mode, 
                               rng=rng, 
                               rng_dim=rand_idx, 
                               rng_width=bitwidth).to(device)
            inputSRC = SourceGen(input, bitwidth, mode=mode, rtype=rtype)().to(device)
            inputRNG = RNG(bitwidth, rand_idx, rng, rtype)().to(device)
            inputBS = BSGen(inputSRC, inputRNG, stype).to(device)
            
            for i in range(0,2**bitwidth-power_off):
                input_bs = inputBS(torch.tensor([i]))
                inputPE.Monitor(input_bs)

                output_bs = dut_tanh_comb(input_bs)
                outputPE.Monitor(output_bs)
        
        # get the result for different rng
        result_pe = outputPE()[1].cpu().numpy()
        result_pe_total_normal.append(result_pe)        
    
    # get the result for different rng
    result_pe_total_normal = np.array(result_pe_total_normal)
    #######################################################################
    # check the error of all simulation
    #######################################################################
    print("RMSE:{:1.4}".format(math.sqrt(np.mean(result_pe_total_normal**2))))
    print("MAE: {:1.4}".format(np.mean(np.abs(result_pe_total_normal))))
    print("bias:{:1.4}".format(np.mean(result_pe_total_normal)))
    print("max: {:1.4}".format(np.max(result_pe_total_normal)))
    print("min: {:1.4}".format(np.min(result_pe_total_normal)))

In [11]:
tanh_comb_test_intermittent_indx(8, "unipolar", "Sobol")

unipolar
RMSE:0.03093
MAE: 0.02412
bias:-0.02316
max: 0.06101
min: -0.1143
